In [1]:
import sys
from sqlglot import parse_one, exp
sys.path.append('../src')

from application.services.sql_utils import replace_literals, to_geospatial_query
from application.services.cosine_similarity import CosineSimilarity
from application.services.embeddings.openai_embedding import OpenAIEmbedding



In [2]:
literals = {
    ("table1", "col2"): ["test1", "test2", "test3"],
    ("table1", "col1"): ["a", "b", "c"],
}

embedding = OpenAIEmbedding()
ts = CosineSimilarity(embedding=embedding)
query = "SELECT a, b, c FROM table1 WHERE col1 = 1 AND col2 = 'ttest1'"

# replace_literals(query, literals, ts)

In [3]:
parsed_query = parse_one("SELECT a1, b2, ST_Union(c) FROM table1 t  WHERE col1 = 1 GROUP BY t.a")

In [11]:
type(parsed_query.find(exp.Select).expressions[0]) == exp.Select


False

In [10]:
to_geospatial_query("SELECT t.a1, t.b2 FROM table1 t  WHERE col1 = 1 GROUP BY t.a", {"table1": "geom2"})

ValueError: Geotable table not found in query.

In [8]:
parsed_query.find(exp.Select).expressions[]

[Column(
   this=Identifier(this=a1, quoted=False)),
 Column(
   this=Identifier(this=b2, quoted=False)),
 Anonymous(
   this=ST_Union,
   expressions=[
     Column(
       this=Identifier(this=c, quoted=False))])]

In [ ]:
exp.Anonymous

In [4]:

parsed_query = parse_one("SELECT a1, b2, HOLA(c) FROM table1 t  WHERE col1 = 1 GROUP BY t.a, t.b")
group_by = parsed_query.find(exp.Group)
xy = [e for e in group_by.expressions]
xy

[Column(
   this=Identifier(this=a, quoted=False),
   table=Identifier(this=t, quoted=False)),
 Column(
   this=Identifier(this=b, quoted=False),
   table=Identifier(this=t, quoted=False))]

In [ ]:
exp.Sum

In [12]:
exp.Literal()

Literal()

In [17]:
parsed_query = parse_one("SELECT a1, b2, HOLA(c) FROM table1 t  WHERE col1 = 1 GROUP BY t.a, t.b")

parsed_query.find(exp.Select).expressions.append("aa")
parsed_query.sql()

'SELECT a1, b2, HOLA(c), aa FROM table1 AS t WHERE col1 = 1 GROUP BY t.a, t.b'

In [ ]:
exp.UserDefinedFunction()

In [20]:
import os
import folium

from infrastructure.postgres_db import PostgresDB


db_name = os.environ.get("DB_NAME")
db_user = os.environ.get("DB_USER")
db_password = os.environ.get("DB_PASSWORD")

if db_name is None or db_user is None or db_password is None:
    raise ValueError("Please set the DB_NAME, DB_USER, and DB_PASSWORD environment variables.")

db = PostgresDB(db_name=db_name, db_user=db_user, db_password=db_password)
gdf = db.run_gpd_query("""SELECT c.nombre AS candidato, p.nombre AS partido, SUM(vm.votos) AS total_votos, ST_Union(com.geom) AS geom 
	FROM candidato AS c JOIN candidatura AS ca ON c.id = ca.candidato_id 
	JOIN partido AS p ON ca.partido_id = p.id JOIN votos_mesa AS vm ON ca.id = vm.candidatura_id 
 JOIN mesa AS m ON vm.mesa_id = m.id JOIN comuna AS com ON m.comuna_id = com.id JOIN eleccion AS e ON ca.eleccion_id = e.id JOIN tipo_eleccion AS te ON e.tipo_eleccion_id = te.id 
		WHERE com.nombre = 'Paredones' AND e.nombre = 'Municipal 2016 - Alcaldes' 
	GROUP BY c.nombre, p.nombre ORDER BY total_votos DESC""")
        
map = folium.Map()
folium.GeoJson(gdf.to_json()).add_to(map)

/Users/jm/Code/mapgpt-mvp/.conda/lib/python3.11/site-packages/geopandas/io/sql.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
/Users/jm/Code/mapgpt-mvp/.conda/lib/python3.11/site-packages/geopandas/io/sql.py:473: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(spatial_ref_sys_sql, con)


In [39]:


px.choropleth_mapbox(gdf).show(renderer="browser")

In [36]:
import plotly.express as px
fig = px.choropleth(gdf, geojson=gdf.to_json(), color='candidato')
fig.show(renderer="browser")

In [32]:
gdf

,candidato,partido,total_votos,geom
0,Sammy Ormazabal Lopez,Partido Por La Democracia,2251,"POLYGON ((-8020724.051 -4102675.677, -8020718...."
1,Emiliano Guerrero Torres,Movimiento Independiente Regionalista Agrario ...,197,"POLYGON ((-8020724.051 -4102675.677, -8020718...."


In [27]:
gdf

,candidato,partido,total_votos,geom
0,Sammy Ormazabal Lopez,Partido Por La Democracia,2251,"POLYGON ((-8020724.051 -4102675.677, -8020718...."
1,Emiliano Guerrero Torres,Movimiento Independiente Regionalista Agrario ...,197,"POLYGON ((-8020724.051 -4102675.677, -8020718...."


In [21]:
map

In [13]:
xy[0].is_column

AttributeError: 'Column' object has no attribute 'is_column'

In [37]:
new_column = exp.Column(this=exp.Identifier(this='email'), table=exp.Identifier(this='a'))

parsed_query.find(exp.Select).expressions.append(new_column)

parsed_query.sql()

'SELECT a1, b2, c, email FROM table1 AS t WHERE col1 = 1 GROUP BY t.a'

In [34]:
xx = parsed_query.find(exp.Group)
str(xx)

'SELECT a1, b2, c,  FROM table1 AS t WHERE col1 = 1 GROUP BY t.a'

In [ ]:
parsed_query.find(exp.Group)

In [56]:
parsed_query.sql()

'SELECT aa, t.b FROM table, table2 AS tt WHERE column = 1'

In [ ]:
[t.table for t in parsed_query.find_all(exp.Column)]

In [52]:
[t.name for t in parsed_query.find_all(exp.Table)]

['table', 'table2']

In [46]:
[t.table for t in parsed_query.find_all(exp.Column)]

['t', 't', '']

In [25]:
parsed_query.find(exp.Literal)

Literal(this=1, is_string=False)

In [42]:
{t.alias:t.name for t in parsed_query.find_all(exp.Table)}

{'t': 'table'}